# Champion Analysis v1.4 - Lightning Prototype

*This notebook is a prototype for the Champion Analysis notebook, intended to be a data collection notebook. Though at the time of creation, it was not intended to be used in the final product, it may be used as a reference for the final product.*

In [1]:
# Dependency Center
import pandas as pd
import numpy as np
import json
import requests
from pprint import pprint

In [2]:
# Test JSON response and display
response = requests.get("https://pokeapi.co/api/v2/pokemon/1").json()
pprint(response)

{'abilities': [{'ability': {'name': 'overgrow',
                            'url': 'https://pokeapi.co/api/v2/ability/65/'},
                'is_hidden': False,
                'slot': 1},
               {'ability': {'name': 'chlorophyll',
                            'url': 'https://pokeapi.co/api/v2/ability/34/'},
                'is_hidden': True,
                'slot': 3}],
 'base_experience': 64,
 'forms': [{'name': 'bulbasaur',
            'url': 'https://pokeapi.co/api/v2/pokemon-form/1/'}],
 'game_indices': [{'game_index': 153,
                   'version': {'name': 'red',
                               'url': 'https://pokeapi.co/api/v2/version/1/'}},
                  {'game_index': 153,
                   'version': {'name': 'blue',
                               'url': 'https://pokeapi.co/api/v2/version/2/'}},
                  {'game_index': 153,
                   'version': {'name': 'yellow',
                               'url': 'https://pokeapi.co/api/v2/version/3/'}},


In [3]:
# Pokemon Information Collection
# Create a dictionary to store the Pokemon's information
pokemon_data = {
    "name": [],
    "type": [],
    "hp": [],
    "attack": [],
    "defense": [],
    "special_attack": [],
    "special_defense": [],
    "speed": [],
    "height": [],
    "weight": [],
    "legendary": [],
    "mythical": []
}

# Loop through all Pokemon from 1 to 898 (Generation 1 to 8)
for i in range(1, 899):
    # Request Pokemon Stats information
    response = requests.get(f"https://pokeapi.co/api/v2/pokemon/{i}").json()

    # Request Pokemon Species information (to get Legendary and Mythical status)
    species_response = requests.get(f"https://pokeapi.co/api/v2/pokemon-species/{i}").json()

    # Pokemon Names
    pokemon_data["name"].append(response["name"])

    # Pokemon Types
    types = [type_info["type"]["name"] for type_info in response["types"]]
    pokemon_data["type"].append(types)

    # Pokemon Stats
    stats = {stat_info["stat"]["name"]: stat_info["base_stat"] for stat_info in response["stats"]}
    pokemon_data["hp"].append(stats.get("hp"))
    pokemon_data["attack"].append(stats.get("attack"))
    pokemon_data["defense"].append(stats.get("defense"))
    pokemon_data["special_attack"].append(stats.get("special-attack"))
    pokemon_data["special_defense"].append(stats.get("special-defense"))
    pokemon_data["speed"].append(stats.get("speed"))

    # Pokemon Measurements
    pokemon_data["height"].append(response["height"])
    pokemon_data["weight"].append(response["weight"])

    # Pokemon Legendary and Mythical Status
    pokemon_data["legendary"].append(species_response["is_legendary"])
    pokemon_data["mythical"].append(species_response["is_mythical"])


# Create a DataFrame from the dictionary
pokemon_df = pd.DataFrame(pokemon_data)


In [4]:
# Display raw DataFrame before refinement
pokemon_df

,name,type,hp,attack,defense,special_attack,special_defense,speed,height,weight,legendary,mythical
0,bulbasaur,"[grass, poison]",45,49,49,65,65,45,7,69,False,False
1,ivysaur,"[grass, poison]",60,62,63,80,80,60,10,130,False,False
2,venusaur,"[grass, poison]",80,82,83,100,100,80,20,1000,False,False
3,charmander,[fire],39,52,43,60,50,65,6,85,False,False
4,charmeleon,[fire],58,64,58,80,65,80,11,190,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
893,regieleki,[electric],80,100,50,100,50,200,12,1450,True,False
894,regidrago,[dragon],200,100,50,100,50,80,21,2000,True,False
895,glastrier,[ice],100,145,130,65,110,30,22,8000,True,False
896,spectrier,[ghost],100,65,60,145,80,130,20,445,True,False


In [5]:
# Fix height and weight units
# Convert decimeters to inches (1 decimeter = 3.93701 inches)
pokemon_df['height'] = (pokemon_df['height'] * 3.93701).round(2)
# Convert hectograms to pounds (1 hectogram = 0.220462 pounds)
pokemon_df['weight'] = (pokemon_df['weight'] * 0.220462).round(2)

In [6]:
# Separate Types into Primary and Secondary
pokemon_df["primary_type"] = pokemon_df["type"].apply(lambda x: x[0])
pokemon_df["secondary_type"] = pokemon_df["type"].apply(lambda x: x[1] if len(x) > 1 else None)

# Capitalize Pokemon Names and Types
pokemon_df["name"] = pokemon_df["name"].str.capitalize()
pokemon_df["primary_type"] = pokemon_df["primary_type"].str.capitalize()
pokemon_df["secondary_type"] = pokemon_df["secondary_type"].str.capitalize()

In [7]:
# Drop Unnecessary Columns and Reorder
pokemon_df = pokemon_df[["name", "primary_type", "secondary_type", "hp", "attack", "defense", "special_attack", "special_defense", "speed", "height", "weight", "legendary", "mythical"]]

# Display changes before refinement
pokemon_df

,name,primary_type,secondary_type,hp,attack,defense,special_attack,special_defense,speed,height,weight,legendary,mythical
0,Bulbasaur,Grass,Poison,45,49,49,65,65,45,27.56,15.21,False,False
1,Ivysaur,Grass,Poison,60,62,63,80,80,60,39.37,28.66,False,False
2,Venusaur,Grass,Poison,80,82,83,100,100,80,78.74,220.46,False,False
3,Charmander,Fire,None,39,52,43,60,50,65,23.62,18.74,False,False
4,Charmeleon,Fire,None,58,64,58,80,65,80,43.31,41.89,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
893,Regieleki,Electric,None,80,100,50,100,50,200,47.24,319.67,True,False
894,Regidrago,Dragon,None,200,100,50,100,50,80,82.68,440.92,True,False
895,Glastrier,Ice,None,100,145,130,65,110,30,86.61,1763.70,True,False
896,Spectrier,Ghost,None,100,65,60,145,80,130,78.74,98.11,True,False


In [8]:
# Rename all columns
pokemon_df = pokemon_df.rename(columns={"name": "Name",
                                        "primary_type": "Primary Type",
                                        "secondary_type": "Secondary Type",
                                        "hp": "HP",
                                        "attack": "Attack",
                                        "defense": "Defense",
                                        "special_attack": "Sp. Atk",
                                        "special_defense": "Sp. Def",
                                        "speed": "Speed",
                                        "height": "Height (in)",
                                        "weight": "Weight (lbs)",
                                        "legendary": "Legendary",
                                        "mythical": "Mythical"})
# Display DataFrame after refinement
pokemon_df


,Name,Primary Type,Secondary Type,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Height (in),Weight (lbs),Legendary,Mythical
0,Bulbasaur,Grass,Poison,45,49,49,65,65,45,27.56,15.21,False,False
1,Ivysaur,Grass,Poison,60,62,63,80,80,60,39.37,28.66,False,False
2,Venusaur,Grass,Poison,80,82,83,100,100,80,78.74,220.46,False,False
3,Charmander,Fire,None,39,52,43,60,50,65,23.62,18.74,False,False
4,Charmeleon,Fire,None,58,64,58,80,65,80,43.31,41.89,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
893,Regieleki,Electric,None,80,100,50,100,50,200,47.24,319.67,True,False
894,Regidrago,Dragon,None,200,100,50,100,50,80,82.68,440.92,True,False
895,Glastrier,Ice,None,100,145,130,65,110,30,86.61,1763.70,True,False
896,Spectrier,Ghost,None,100,65,60,145,80,130,78.74,98.11,True,False


In [9]:
# Export new DataFrame to CSV
pokemon_df.to_csv("Extractions/Refined Pokemon Roster.csv", index=False, header=True)

# Export new DataFrame to Excel file for good measure
pokemon_df.to_excel("Extractions/Refined Pokemon Roster.xlsx", index=False, header=True)